#El arte mas allá del humano.

Integrantes:

-  Luis Gerardo Castellano Vergaño - 2142611
- Andrés Camilo Hernández Arias - 2130284
- Ángel Fabian Gómez Estupiñan - 2130535




> Este es un algoritmo desarrollado con Redes Neuronales Recurrentes (RNN) que  genera, a partir de un entrenamiento y de una cancion dada (datos de entrada), una secuencia similiar a la canción de entrada.



##Obtjetivo:
> Facilitar el proceso de composición musical con un algoritmo que genere composiciones de acuerdo a un ritmo musical en forma de datos de entrenamiento, esta composición puede servir de base para una composición musical más avanzada.








In [0]:
#@test {"output": "ignore"}

print 'Installing dependencies...'
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print 'Importing libraries and defining some helper functions...'
from google.colab import files

import magenta.music as mm
import magenta
import tensorflow

print '🎉 Done!'
print magenta.__version__ 
print tensorflow.__version__

Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 131289 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...


In [0]:
from magenta.protobuf import music_pb2

twinkle_twinkle = music_pb2.NoteSequence()

# Add the notes to the sequence.
twinkle_twinkle.notes.add(pitch=60, start_time=0.0, end_time=0.5, velocity=80)
twinkle_twinkle.notes.add(pitch=60, start_time=0.5, end_time=1.0, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=1.0, end_time=1.5, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=1.5, end_time=2.0, velocity=80)
twinkle_twinkle.notes.add(pitch=69, start_time=2.0, end_time=2.5, velocity=80)
twinkle_twinkle.notes.add(pitch=69, start_time=2.5, end_time=3.0, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=3.0, end_time=4.0, velocity=80)
twinkle_twinkle.notes.add(pitch=65, start_time=4.0, end_time=4.5, velocity=80)
twinkle_twinkle.notes.add(pitch=65, start_time=4.5, end_time=5.0, velocity=80)
twinkle_twinkle.notes.add(pitch=64, start_time=5.0, end_time=5.5, velocity=80)
twinkle_twinkle.notes.add(pitch=64, start_time=5.5, end_time=6.0, velocity=80)
twinkle_twinkle.notes.add(pitch=62, start_time=6.0, end_time=6.5, velocity=80)
twinkle_twinkle.notes.add(pitch=62, start_time=6.5, end_time=7.0, velocity=80)
twinkle_twinkle.notes.add(pitch=60, start_time=7.0, end_time=8.0, velocity=80) 
twinkle_twinkle.total_time = 8

twinkle_twinkle.tempos.add(qpm=60);

# This is a colab utility method that visualizes a NoteSequence.
mm.plot_sequence(twinkle_twinkle)

# This is a colab utility method that plays a NoteSequence.
mm.play_sequence(twinkle_twinkle,synth=mm.fluidsynth)

# Here's another NoteSequence!
teapot = music_pb2.NoteSequence()
teapot.notes.add(pitch=69, start_time=0, end_time=0.5, velocity=80)
teapot.notes.add(pitch=71, start_time=0.5, end_time=1, velocity=80)
teapot.notes.add(pitch=73, start_time=1, end_time=1.5, velocity=80)
teapot.notes.add(pitch=74, start_time=1.5, end_time=2, velocity=80)
teapot.notes.add(pitch=76, start_time=2, end_time=2.5, velocity=80)
teapot.notes.add(pitch=81, start_time=3, end_time=4, velocity=80)
teapot.notes.add(pitch=78, start_time=4, end_time=5, velocity=80)
teapot.notes.add(pitch=81, start_time=5, end_time=6, velocity=80)
teapot.notes.add(pitch=76, start_time=6, end_time=8, velocity=80)
teapot.total_time = 8

teapot.tempos.add(qpm=60);

mm.plot_sequence(teapot)
mm.play_sequence(teapot,synth=mm.synthesize)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf
from magenta.music import midi_io
#('/content/drive/My Drive/Coolio_-_Gangstas_Paradise.mid')
#tf.io.GFile.GFile('/content/drive/My Drive/Coolio_-_Gangstas_Paradise.mid')
ganstas_paradise = midi_io.midi_file_to_note_sequence('/content/drive/My Drive/coro_edited.mid')

In [0]:
# This is a colab utility method that visualizes a NoteSequence.
mm.plot_sequence(ganstas_paradise)

# This is a colab utility method that plays a NoteSequence.
mm.play_sequence(ganstas_paradise,synth=mm.fluidsynth)

In [0]:
print 'Downloading model bundle. This will take less than a minute...'
mm.notebook_utils.download_bundle('basic_rnn.mag', '/content/')
bundle = mm.sequence_generator_bundle.read_bundle_file('/content/basic_rnn.mag')

# Import dependencies.
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.protobuf import generator_pb2
from magenta.protobuf import music_pb2

# Initialize the model.
print "Initializing Melody RNN..."
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['basic_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

print '🎉 Done!'

W0818 14:24:23.312266 140161642420096 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/magenta/music/notebook_utils.py:196: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.



W0818 14:24:27.213205 140161642420096 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0818 14:24:27.254260 140161642420096 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/magenta/music/model.py:70: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0818 14:24:27.285957 140161642420096 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/magenta/music/model.py:71: The name tf.train.import_meta_graph is deprecated. Please use tf.compat.v1.train.import_meta_graph instead.

W0818 14:24:27.355214 140161642420096 meta_graph.py:935] The saved meta_graph is possibly from an older release:
'mode

Initializing Melody RNN...


W0818 14:24:27.564529 140161642420096 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/magenta/music/sequence_generator.py:159: The name tf.gfile.DeleteRecursively is deprecated. Please use tf.io.gfile.rmtree instead.



🎉 Done!


In [0]:
# Model options. Change these to get different generated sequences! 

input_sequence = ganstas_paradise # change this to teapot if you want
num_steps = 200 # change this for shorter or longer sequences
temperature = 1.5 # the higher the temperature the more random the sequence.

# Set the start time to begin on the next step after the last note ends.
last_end_time = (max(n.end_time for n in input_sequence.notes)
                  if input_sequence.notes else 0)
qpm = input_sequence.tempos[0].qpm 
seconds_per_step = 60.0 / qpm / melody_rnn.steps_per_quarter
total_seconds = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(
  start_time=last_end_time + seconds_per_step,
  end_time=total_seconds)

# Ask the model to continue the sequence.
sequence = melody_rnn.generate(input_sequence, generator_options)

mm.plot_sequence(sequence)
mm.play_sequence(sequence, synth= mm.fluidsynth)

In [0]:
# This creates a file called `ganstas_RNN1.mid`
mm.sequence_proto_to_midi_file(sequence, 'ganstas_RNN1.mid')

# This is a colab utility method to download that file. In your Python script, you 
# would just write it to disk.
files.download('ganstas_RNN1.mid')